In [1]:
from joblib import dump, load
import pandas as pd
import numpy as np
from collections import Counter

In [2]:

@np.vectorize
def estimate_bin(m, n, iterations=100, alpha=0.05):
    try:
        prob_ests = np.random.binomial(n, m / n, iterations) / n
        low = np.quantile(prob_ests, alpha/2)
        up = np.quantile(prob_ests, 1-alpha/2)
    except:
        return np.nan
    #return f'({np.round(low,3)}, {np.round(up,4)})'
    return np.round(m/n, 3)


In [3]:
feature_intervals = {
    'aspect': np.arange(0, 370, 10),
    'slope' :  np.arange(0, 70, 2),
    #'forest-height': np.arange(0, 60, 1),
    'elevation': np.append(np.arange(0, 650, 10),2000),
    'tree_cover': np.arange(20, 105, 2),
    'curvature': np.arange(-20, 20, 0.5),
    # 'curvature-plan': np.arange(-13, 13, 0.5),
    # 'curvature-prof': np.arange(-13, 13, 0.5),
    'patches': np.array([0, 10, 100, 1000, 10000, 100000, 1000000, 1500000, 2000000]),
    #'patches': np.array([0] + [np.exp(j) for j in range(1, 16)]),
    'morphology': np.arange(0, 12, 1),
    'vegetation': []
}


In [4]:
features = load('output_veg_4M.dat')

In [5]:
features=pd.DataFrame(features)

In [6]:
features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000000 entries, 0 to 3999999
Data columns (total 11 columns):
 #   Column      Dtype  
---  ------      -----  
 0   aspect      float64
 1   curvature   float64
 2   elevation   float64
 3   morphology  float64
 4   slope       float64
 5   tree_cover  float64
 6   vegetation  float64
 7   lats        float64
 8   lons        float64
 9   patches     float64
 10  wfall       bool   
dtypes: bool(1), float64(10)
memory usage: 309.0 MB


In [7]:
((features.vegetation == 2) * features.wfall).sum()

2851

In [8]:
features.dropna(axis=0, inplace=True)

In [9]:
features = features[(features.elevation != 32767) & (features['aspect'] != -1) & (~np.isnan(features['morphology'])) & (features['tree_cover'] > 20) & (features['tree_cover'] <= 100)]

In [10]:
total = sum(v for k, v in Counter(features.vegetation).items() if k != 2)
probs = [v / total for k, v in Counter(features.vegetation).items() if k!=2]
vals = [k for k, v in Counter(features.vegetation).items() if k!=2]
features.loc[features.vegetation == 2, 'vegetation'] = np.random.choice(vals, size=((features.vegetation == 2).sum(),), p=probs)

In [11]:
features.shape

(511053, 11)

In [12]:
features[features.wfall].shape

(9091, 11)

In [13]:
for name in feature_intervals:
    if name != 'vegetation':
        gr_name = np.digitize(features[name], bins=feature_intervals[name])
        features['gr_'+name] = list(map(lambda i: f"{feature_intervals[name][i-1]}<={name}<{feature_intervals[name][i]}",gr_name))
    else:
        features['gr_'+name] = features[name]
    


In [14]:
features

,aspect,curvature,elevation,morphology,slope,tree_cover,vegetation,lats,lons,patches,wfall,gr_aspect,gr_slope,gr_elevation,gr_tree_cover,gr_curvature,gr_patches,gr_morphology,gr_vegetation
15,329.036255,-0.3888,404.0,6.0,6.740799,90.0,6.0,4.916115e+06,438098.928341,0.0,False,320<=aspect<330,6<=slope<8,400<=elevation<410,90<=tree_cover<92,-0.5<=curvature<0.0,0<=patches<10,6<=morphology<7,6.0
16,205.346176,-0.3888,234.0,7.0,4.871637,85.0,7.0,4.907124e+06,437201.617039,0.0,False,200<=aspect<210,4<=slope<6,230<=elevation<240,84<=tree_cover<86,-0.5<=curvature<0.0,0<=patches<10,7<=morphology<8,7.0
32,77.691986,-0.0000,586.0,5.0,24.533701,56.0,3.0,4.913983e+06,428869.354517,0.0,False,70<=aspect<80,24<=slope<26,580<=elevation<590,56<=tree_cover<58,0.0<=curvature<0.5,0<=patches<10,5<=morphology<6,3.0
34,60.945396,0.5184,16.0,7.0,2.390084,85.0,21.0,4.903206e+06,428996.060925,0.0,False,60<=aspect<70,2<=slope<4,10<=elevation<20,84<=tree_cover<86,0.5<=curvature<1.0,0<=patches<10,7<=morphology<8,21.0
45,0.000000,-0.1296,14.0,6.0,5.096679,58.0,14.0,4.877000e+06,404082.976082,0.0,False,0<=aspect<10,4<=slope<6,10<=elevation<20,58<=tree_cover<60,-0.5<=curvature<0.0,0<=patches<10,6<=morphology<7,14.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3999950,91.847610,-0.6480,200.0,8.0,7.166818,85.0,3.0,4.907707e+06,421565.779741,0.0,False,90<=aspect<100,6<=slope<8,200<=elevation<210,84<=tree_cover<86,-1.0<=curvature<-0.5,0<=patches<10,8<=morphology<9,3.0
3999953,187.011856,-1.0368,258.0,5.0,37.373344,97.0,6.0,4.911949e+06,424488.277351,0.0,False,180<=aspect<190,36<=slope<38,250<=elevation<260,96<=tree_cover<98,-1.5<=curvature<-1.0,0<=patches<10,5<=morphology<6,6.0
3999966,225.000000,1.1664,95.0,2.0,1.642019,86.0,7.0,4.871964e+06,391823.200119,0.0,False,220<=aspect<230,0<=slope<2,90<=elevation<100,86<=tree_cover<88,1.0<=curvature<1.5,0<=patches<10,2<=morphology<3,7.0
3999972,348.690063,1.1664,77.0,7.0,8.233659,90.0,7.0,4.866273e+06,392486.010892,0.0,False,340<=aspect<350,8<=slope<10,70<=elevation<80,90<=tree_cover<92,1.0<=curvature<1.5,0<=patches<10,7<=morphology<8,7.0


In [15]:
for name in feature_intervals:
    patch_result = features.groupby(['gr_' + name, 'gr_patches', 'wfall'])[[name]].count().unstack(level=0)
    patch_result = (patch_result * (4 * 10 ** 6 / patch_result.sum().sum())).fillna(0).astype(np.int64)
    patch_result.iloc[0,0] += 4 * 10 ** 6 - patch_result.sum().sum() 
    patch_result.to_csv(f"patched_cnt_{name}.csv")
    #patch_result.T.apply(lambda x: estimate_bin(x, x.fillna(0).sum())).T.to_csv(f"patched_int_{name}.csv")
    total_result = features.groupby(['gr_'+ name, 'wfall'])[[name]].count().unstack(level=0)
    total_result = (total_result * (4 * 10 ** 6 / total_result.sum().sum())).fillna(0).astype(np.int64)
    total_result.iloc[0,0] += 4 * 10 ** 6 - total_result.sum().sum() 
    total_result.to_csv(f"total_cnt_{name}.csv")
    print(total_result.sum().sum(), patch_result.sum().sum(), name)
    #total_result.T.apply(lambda x: estimate_bin(x, x.fillna(0).sum())).T.to_csv(f"total_int_{name}.csv")


4000000 4000000 aspect
4000000 4000000 slope
4000000 4000000 elevation
4000000 4000000 tree_cover
4000000 4000000 curvature
4000000 4000000 patches
4000000 4000000 morphology
4000000 4000000 vegetation


In [16]:
patch_result

vegetation                                   \
gr_vegetation                      0.0      3.0    4.0      6.0     7.0     
gr_patches              wfall                                               
0<=patches<10           False      88332  1166477  23974  1404284  512666   
100000<=patches<1000000 True          70     1620     23     2066    3240   
10000<=patches<100000   True         187     4461     31     5846    5713   
1000<=patches<10000     True         297     6919     70     9032    7709   
100<=patches<1000       True         117     3890     23     6692    4797   
10<=patches<100         True          23      407      0      892     532   

                                                                          \
gr_vegetation                  9.0    11.0    14.0    16.0  18.0   20.0    
gr_patches              wfall                                              
0<=patches<10           False  25946  87615  236045  232046  5024  16898   
100000<=patches<1000000 True      15    109     305     234    15     31   
10000<=patches<100000   True      54    234     814     500    15     54   
1000<=patches<10000     True      46    164     947     407    15     54   
100<=patches<1000       True      15     62     289     156     0     15   
10<=patches<100         True       0     15      23       7     0      0   

                                             
gr_vegetation                   21.0  127.0  
gr_patches              wfall                
0<=patches<10           False  128080  1487  
100000<=patches<1000000 True      101     0  
10000<=patches<100000   True      469     7  
1000<=patches<10000     True      790     7  
100<=patches<1000       True      469     0  
10<=patches<100         True       31     0

In [17]:
pd.pivot_table(features, 'wfall',index=features.wfall.values, columns=['vegetation'],  aggfunc='count')

vegetation,0.0,3.0,4.0,6.0,7.0,9.0,11.0,14.0,16.0,18.0,20.0,21.0,127.0
False,11281,149033,3063,179416,65500,3315,11194,30158,29647,642,2159,16364,190
True,89,2210,19,3134,2810,17,75,304,167,6,20,238,2


In [89]:
cols = list(feature_intervals)
cols.remove('patches')

In [90]:
cols

['aspect',
 'slope',
 'elevation',
 'tree_cover',
 'curvature',
 'morphology',
 'vegetation']

In [29]:
y = features.wfall.values
X = features[cols].values

In [30]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
clf.fit(X, y)
clf.feature_importances_

array([0.23868993, 0.23083277, 0.2214276 , 0.07105296, 0.1479691 ,
       0.04941433, 0.04061331])

In [31]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import balanced_accuracy_score
print(cross_val_score(clf, X, y, cv=3, scoring='accuracy'))

[0.98235408 0.98242452 0.98250084]


# Model building (RF classifier)

In [32]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from collections import Counter

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis, LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix
import seaborn as sns
from joblib import dump, load

In [33]:
estimators = [
    ('rfc', RandomForestClassifier()),
#     ('lda', LinearDiscriminantAnalysis()),
#     ('qda', QuadraticDiscriminantAnalysis()),
#     ('log', LogisticRegression())
]

params = {
    'rfc': {
        'n_estimators': [200],
        'max_depth': [20],
        'min_samples_split': [2000],
        'class_weight': ['balanced_subsample']
    },
    
    'lda': {
        'n_components': [2,]
    },
    
    'qda': {
        
        
    },
    
    'log': {
        'penalty' : ['l1', 'l2'],
        'C': [1.0, 0.1, 0.01, 0.001],
        'class_weight': ['balanced']
    }
    
}

In [34]:
X_ = features[cols].values
y = features.wfall.astype(int)

In [35]:
results = dict()
for cl_name, clf in estimators:
    gcv = GridSearchCV(clf, param_grid=params[cl_name], n_jobs=14, scoring='balanced_accuracy', verbose=1, cv=5)
    gcv.fit(X_, y)
    print(f"Resutls for {cl_name}.")
    print(f"Balanced accuracy score: {gcv.best_score_}")
    print(f"Confusion matrix: {confusion_matrix(gcv.best_estimator_.predict(X_), y)}")
    results[cl_name] = gcv

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=14)]: Using backend LokyBackend with 14 concurrent workers.
[Parallel(n_jobs=14)]: Done   2 out of   5 | elapsed:  2.4min remaining:  3.6min
[Parallel(n_jobs=14)]: Done   5 out of   5 | elapsed:  2.5min finished


Resutls for rfc.
Balanced accuracy score: 0.7309025771647258
Confusion matrix: [[361896   1745]
 [140066   7346]]


In [36]:
for f, imp in zip(cols, results['rfc'].best_estimator_.feature_importances_):
    print(f"feature {f}: {imp}.")

feature aspect: 0.12221288066649377.
feature slope: 0.08769877374793465.
feature elevation: 0.48090052071742967.
feature tree_cover: 0.1358940495667604.
feature curvature: 0.0250195698720665.
feature morphology: 0.02458790455940345.
feature vegetation: 0.12368630086991145.


In [37]:
n_features = len(cols)

In [38]:
results = []
clf = RandomForestClassifier(**{'class_weight': 'balanced_subsample',
 'max_depth': 20,
 'min_samples_split': 4000,
 'n_estimators': 10})
clf.fit(X_, y)

RandomForestClassifier(class_weight='balanced_subsample', max_depth=20,
                       min_samples_split=4000, n_estimators=10)

In [39]:
dump(clf, 'rf_clf_kun.joblib') 

['rf_clf_kun.joblib']

In [40]:
results = []
clf = RandomForestClassifier(**{'class_weight': 'balanced_subsample',
 'max_depth': 20,
 'min_samples_split': 4000,
 'n_estimators': 10})
for j in range(2, 2 ** n_features - 1):
    print(j)
    mask = np.array(list(format(j, f'#0{n_features + 2}b')[2:]), dtype=int).astype(bool)
    if sum(mask) > 2:
        errors = cross_val_score(clf, X_[:, mask], y, n_jobs=14,scoring='balanced_accuracy')
        results.append((mask, errors))
        # print(f"Trying features {np.array(features)[mask]}: mean={errors.mean()}, std={errors.std()}.")

2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126


In [41]:
for ffs, vals in sorted(results, key=lambda x: x[1].mean(), reverse=True):
    print(np.array(cols)[ffs], vals.mean())

['aspect' 'slope' 'elevation' 'tree_cover' 'morphology' 'vegetation'] 0.725798562713009
['aspect' 'slope' 'elevation' 'tree_cover' 'curvature' 'vegetation'] 0.7255991594523992
['aspect' 'elevation' 'tree_cover' 'morphology' 'vegetation'] 0.7230929963559547
['aspect' 'slope' 'elevation' 'tree_cover' 'vegetation'] 0.7227537864955914
['aspect' 'elevation' 'tree_cover' 'curvature' 'morphology' 'vegetation'] 0.721638886013578
['aspect' 'elevation' 'tree_cover' 'vegetation'] 0.7196286685122805
['aspect' 'elevation' 'tree_cover' 'curvature' 'vegetation'] 0.7182728433325065
['slope' 'elevation' 'tree_cover' 'vegetation'] 0.7161274819971523
['aspect' 'slope' 'elevation' 'morphology' 'vegetation'] 0.7156351604065632
['slope' 'elevation' 'tree_cover' 'morphology' 'vegetation'] 0.714665204290214
['elevation' 'tree_cover' 'morphology' 'vegetation'] 0.7139164094147243
['slope' 'elevation' 'tree_cover' 'curvature' 'vegetation'] 0.7137977545188565
['slope' 'elevation' 'tree_cover' 'curvature' 'morphol

In [67]:
comm = []
for ffs, vals in filter(lambda x: x[1].mean()>0.71,results):
    comm.append(np.array(cols)[ffs].tolist())

In [68]:
from functools import reduce

In [69]:
reduce(lambda x, y: x.intersection(y), map(set, comm))

{'elevation', 'vegetation'}

In [117]:
data = total_result.T.apply(lambda x: estimate_bin(x, x.fillna(0).sum())).T.to_numpy()
data[~np.isnan(data)].sum()

C:\Users\ASUS\anaconda3\envs\sci\lib\site-packages\numpy\lib\function_base.py:2192: RuntimeWarning: invalid value encountered in ? (vectorized)
  outputs = ufunc(*inputs)


1.9979999999999998

In [156]:
total_result

vegetation                                                 \
gr_vegetation      0.0      3.0    4.0      6.0     7.0    9.0    11.0    
wfall                                                                     
False              88264  1166106  23974  1405050  512487  25821  87536   
True                 665    17477    133    24654   21931     78    453   

                                                          
gr_vegetation   14.0    16.0  18.0   20.0    21.0  127.0  
wfall                                                     
False          235858  232351  4993  16961  127978  1471  
True             2230    1416    46    140    1925     0